In [ ]:
# See requirement.txt

In [ ]:
import glob
import numpy as np
import cv2
import os
import keras 
from keras import backend as K
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ReLU
from matplotlib import pyplot as plt
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Dimensions of our images.
img_width, img_height = 61, 61

train_data_dir = './training'
validation_data_dir = './validation'

# define input shape
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_width, img_height)
else:
    input_shape = (img_width, img_height, 1)

In [ ]:
# Build model 
model = Sequential() 
model.add(Conv2D(32, kernel_size=(6, 6),activation='relu',input_shape=input_shape,padding='valid')) 
#model.add(Dropout(0.2)) 
model.add(MaxPooling2D((2, 2),padding='same')) 
model.add(Conv2D(64, (5, 5), activation='relu',padding='valid')) 
#model.add(Dropout(0.2)) 
model.add(MaxPooling2D(pool_size=(2, 2),padding='same', )) 
model.add(Conv2D(128, (4, 4), activation='relu',padding='valid')) 
model.add(MaxPooling2D(pool_size=(2, 2),padding='same')) 
model.add(Flatten()) 
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.8)) 
model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='binary_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])

In [ ]:
# Use ImageDataGenerator
# Augment input using vertical and horizontal flip
train_datagen = ImageDataGenerator(
  rescale = 1./255, # scaling
  vertical_flip = True,
  horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary')


In [ ]:
model_train_val = model.fit_generator(generator=train_generator,validation_data=validation_generator, steps_per_epoch = 3000, validation_steps= 200, epochs = 33, use_multiprocessing=False, max_queue_size=10, workers=4, verbose=1)

In [ ]:
accuracy = model_train_val.history['acc']
val_accuracy = model_train_val.history['val_acc']
loss = model_train_val.history['loss']
val_loss = model_train_val.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'C1', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('acc.png')

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'C1', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
#plt.show()
plt.savefig('loss.png')

In [ ]:
# Save model
from keras.models import model_from_json
from keras.models import load_model
model_json = model_train_val.model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
model_train_val.model.save_weights("model.h5")
print("Saved model")